In [66]:
import pandas as pd
import numpy as np
import scipy as sp
from statistics import mode

from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier

In [2]:
base_path = '../data/e-dom/data/defect'

In [3]:
data_fname = 'ant.csv'

In [4]:
df = pd.read_csv(f'{base_path}/{data_fname}')

In [5]:
x = df.drop('bug', axis=1)
y = df['bug']

In [39]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
x_train, x_test, y_train, y_test = map(np.array, (x_train, x_test, y_train, y_test))

In [40]:
type(x_train)

numpy.ndarray

In [46]:
# from https://stackoverflow.com/a/35458920
def dbscan_predict(dbscan_model, X_new, metric=sp.spatial.distance.cosine):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS
        for i, x_core in enumerate(dbscan_model.components_): 
            if metric(x_new, x_core) < dbscan_model.eps:
                # Assign label of x_core to x_new
                y_new[j] = dbscan_model.labels_[dbscan_model.core_sample_indices_[i]]
                break

    return y_new

In [248]:
dbs = DBSCAN(min_samples=2, eps=701)

In [249]:
preds = dbs.fit_predict(x_train)

In [250]:
np.unique(preds)

array([-1,  0,  1])

In [251]:
len(np.where(preds == -1)[0])

5

In [252]:
len(x_train)

558

It's about 10% now.

In [253]:
test_preds = dbscan_predict(dbs, np.array(x_test), metric=sp.spatial.distance.euclidean)

In [254]:
# Find the class which is cluster 0
idx = np.where(preds == 0)[0]
mode(y_train[idx])

0

In [255]:
test_preds

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [256]:
c0_idx = np.where(test_preds == 0)[0]
sum(y_test[c0_idx]), len(c0_idx)

(42, 186)

In [257]:
c1_idx = np.where(test_preds == 1)[0]
sum(y_test[c1_idx]), len(c1_idx)

(0, 0)

In [258]:
len(x_test)

187

* We predict all of cluster 0 as class 0
* 186 of the test set has predicted label 0
* 42 of those are wrong (false negatives)
* 0 of the test set has predicted label 1.

In [106]:
169 + 1 + len(np.where(test_preds == -1)[0])

187

In [259]:
# Now do a 1-NN for the -1 points
tree = sp.spatial.cKDTree(x_train)
c_1_preds = []
test_set = x_test[np.where(test_preds == -1)[0]]
for i, sample in enumerate(test_set):
    result = tree.query(sample, k=3)
    _preds = y_train[result[1]]
    c_1_preds.append(mode(_preds))
    #c_1_preds.append(y_train[result[1]])

In [260]:
c_1_preds

[1]

In [261]:
c_1_0_idx = np.where(np.array(c_1_preds) == 0)[0]
sum(y_test[c_1_0_idx]), len(c_1_0_idx)

(0, 0)

In [262]:
c_1_1_idx = np.where(np.array(c_1_preds) == 1)[0]
sum(y_test[c_1_1_idx]), len(c_1_1_idx)

(1, 1)

* 0 of the test set has predicted label 0, and 0 of those is wrong
* 1 of the test set has predicted label 1, and 1 of those are right

To summarize:

* Pred = 0, Target = 0:  144
* Pred = 0, Target = 1:  42
* Pred = 1, Target = 0:  0
* Pred = 1, Target = 1:  1

In [263]:
print('Recall =', 1 / 43)

Recall = 0.023255813953488372


In [264]:
print('Precision =', 1 / 1)

Precision = 1.0


In [265]:
print('False alarm =', 0 / 144)

False alarm = 0.0


In [85]:
print('Accuracy =', 142 / 187)

Accuracy = 0.7593582887700535
